# 데이터 전처리 이해와 실무

## 데이터변환 : 구간화, 정규화

## 구간화, 정규화 (강의 교안)

1. 변수 변환 방안
    - 구간화 : 연속형 데이터의 범주화
    - 정규화 : 연속형 데이터들의 관측 간격 조정 (스케일링)
    
## 실습 내용 요약
    - 예제 데이터 활용 기반 구간화/ 정규화 방안 실습
    
## 실습 데이터
    - 데이터 변환 강좌 실습을 위한 생성 데이터 : 기사 별 클릭 수 데이터

## 실습 시 활용 패키지
    - pandas
    - numpy
    - scikit-learn


In [1]:
import pandas as pd
import numpy as np

In [11]:
# 데이터 로딩 및 개요 확인
# 실습데이터 제공받는다면 그 데이터로 해보기
# click_data = pd.read_csv("./data/click_sample_data.csv", encoding = 'cp949')
# click_data.head

# 실습데이터 임의로 만들기
np.random.seed(1)
cat_list = ['사설', '사회', '공학', '증권', '부동산', '정치', '스포츠']
cat = np.random.randint(0,7, size=10000)

np.random.seed(2)
jour_list = ['A일보', 'B일보', 'C일보', 'D일보', 'E일보']
jour = np.random.randint(0,5, size=10000)

np.random.seed(3)
num_click = np.random.randint(9, 3000, size=10000)

make_data = []
for i in range(10000):
    make_data.append([cat_list[cat[i]], jour_list[jour[i]], num_click[i]])
    
# 이상치 만들기
make_data[10][2] = 9
make_data[9000][2] = 433992


#만든 데이터를 데이터프레임으로
click_data = pd.DataFrame(np.array(make_data))
click_data.columns = ['category', 'Journal', 'num_click']
click_data['num_click'] = click_data['num_click'].astype(float)
click_data

,category,Journal,num_click
0,정치,A일보,1907.0
1,증권,A일보,1697.0
2,부동산,D일보,1676.0
3,사설,C일보,977.0
4,사회,D일보,2313.0
...,...,...,...
9995,사회,B일보,2184.0
9996,정치,D일보,1080.0
9997,사회,A일보,2583.0
9998,사회,B일보,2151.0


In [12]:
# 데이터 개요 파악
click_data.info

#데이터 copy
click_copy = click_data.copy()

## 구간화 (Binning)

### 구간화 방안
    1. 지정 길이 기반 구간 : 데이터 범위를 사용자의 기준대로 정의하고 구간화
    2. 분포 기반 구간 : 각 데이터가 모든 구간 내 동일한 개수로 구분되도록 구간화

In [14]:
click_copy.describe()

,num_click
count,10000.00000
mean,1545.51690
std,4410.74567
min,9.00000
25%,738.75000
50%,1522.00000
75%,2249.00000
max,433992.00000


- 기사 클릭수가 20회인 기사와 10회인 기사 비교
- 20회 클릭된 기사가 2배 더 흥미있는 기사로 정의할 수 있는가?
    -> 없음! 기사의 카테고리, 저널 등 클릭을 유도하는 여러 요소가 있기 때문

- 따라서, 구간화를 통해 새로운 의미를 지닌 변수로 변환이 필요하며 목적 설정이 중요함
- 클릭 횟수를 기준으로 범주로 변환하여 각 범주 별 의미 설정 가능 (도매인 지식 검토 필요)

In [15]:
# 지정 길이 기반 구간화
# 클릭 수 기반을 기사 관심도 정의 기반 범주형 변환 (3개 구간 설정)

# cut 함수 활용 기반 구간 설정(사용자 기준)
bins = [0, 100, 500, np.max(click_copy['num_click'])]
names = ['low', 'medium', 'high']

click_copy['pref'] = pd.cut(click_copy['num_click'], bins, labels = names)

In [16]:
# 구간 결과 확인
click_copy['pref'].value_counts()

high      8321
medium    1366
low        313
Name: pref, dtype: int64

In [17]:
click_copy.head(15)

,category,Journal,num_click,pref
0,정치,A일보,1907.0,high
1,증권,A일보,1697.0,high
2,부동산,D일보,1676.0,high
3,사설,C일보,977.0,high
4,사회,D일보,2313.0,high
5,증권,A일보,798.0,high
6,정치,C일보,2716.0,high
7,사설,B일보,1747.0,high
8,사설,D일보,2932.0,high
9,사회,C일보,1714.0,high


In [18]:
# 분포 기반 구간화
# qcut의 경우 동일한 관측치가 배치되도록 구간을 분할
# 따라서 구간수로 나누게 됨
# 구간수 n
n = 3
click_copy['pref_qcut'] = pd.qcut(click_copy['num_click'], n, labels = names)
print(click_copy['pref_qcut'].value_counts())

medium    3335
low       3334
high      3331
Name: pref_qcut, dtype: int64


#### 만일 qcut 실행 시 주로 발생하는 오류
 - 데이터가 극단치의 Skewed 한 경우 발생
 - 데이터의 각 구간은 동일할 수 없기 때문
     - [1,1,1,1,1,3,8,10]이라는 구간은 생성 불가능
 - 그러한 경우에는 분포 기반구간화(qcut)보다 사용자 기준 기반 범주화 혹은 구간 개수 변화 고려

## 정규화

#### 기계학습 알고리즘의 각 변수별 영향력의 차이를 조정할 필요 존재함
    - 각 변수 별 범위가 다를 경우 학습 안정성이 떨어질 수 있으므로 간격 조정 필요
    
    1. 최대-최소 정규화 : 편수 안의 값을 [0,1] 구간의 값을 갖도록 구성하고 데이터 군 내에서 특정데이터가 가지는 위치를 보고자 할때 사용
    2. z-score 정규화 : 특정 데이터가 평균을 기준으로 얼마나 떨어져 있는지 파악하고 측정 스케일이 다른 경우의 데이터를 비교하는데 사용 (표준화)

In [23]:
# 최대-최소 정규화 [Min-Max Scaling]
# 변수 값을 0에서 1사이의 값으로 변환
# 함수는 sklearn 라이브러리에 쉽게 구현되어있음

from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()

# 데이터 Copy
click_copy = click_data.copy()

# 변환

click_copy['minmax_values'] = scaler.fit_transform(click_copy[['num_click']])
click_copy.head(15)

,category,Journal,num_click,minmax_values
0,정치,A일보,1907.0,0.004373
1,증권,A일보,1697.0,0.003890
2,부동산,D일보,1676.0,0.003841
3,사설,C일보,977.0,0.002231
4,사회,D일보,2313.0,0.005309
5,증권,A일보,798.0,0.001818
6,정치,C일보,2716.0,0.006238
7,사설,B일보,1747.0,0.004005
8,사설,D일보,2932.0,0.006735
9,사회,C일보,1714.0,0.003929


In [25]:
click_copy.describe()

,num_click,minmax_values
count,10000.00000,10000.000000
mean,1545.51690,0.003541
std,4410.74567,0.010163
min,9.00000,0.000000
25%,738.75000,0.001682
50%,1522.00000,0.003486
75%,2249.00000,0.005161
max,433992.00000,1.000000


In [27]:
# Z-Score 정규화 (Z-Score Noramalization) : 표주노하라고 불리우기도 함
# 변수를 평균이 0이고 표준편차가 1 인 정규분포로 변환
# 함수는 sklearn 라이브러리에 쉽게 구현되어있음

from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()

# 변환
click_copy['std_values'] = std_scaler.fit_transform(click_copy[['num_click']])
click_copy.head(15)

,category,Journal,num_click,minmax_values,std_values
0,정치,A일보,1907.0,0.004373,0.081959
1,증권,A일보,1697.0,0.003890,0.034346
2,부동산,D일보,1676.0,0.003841,0.029584
3,사설,C일보,977.0,0.002231,-0.128900
4,사회,D일보,2313.0,0.005309,0.174012
5,증권,A일보,798.0,0.001818,-0.169485
6,정치,C일보,2716.0,0.006238,0.265384
7,사설,B일보,1747.0,0.004005,0.045682
8,사설,D일보,2932.0,0.006735,0.314358
9,사회,C일보,1714.0,0.003929,0.038200


In [28]:
# 데이터 요약
click_copy.describe()
# 이상치 제거후 수치형 데이터를 변환하여 모델링 적용 필요

,num_click,minmax_values,std_values
count,10000.00000,10000.000000,1.000000e+04
mean,1545.51690,0.003541,-1.851679e-17
std,4410.74567,0.010163,1.000050e+00
min,9.00000,0.000000,-3.483750e-01
25%,738.75000,0.001682,-1.829186e-01
50%,1522.00000,0.003486,-5.331995e-03
75%,2249.00000,0.005161,1.595010e-01
max,433992.00000,1.000000,9.804875e+01


### 어느 방안이 더 나은것인가?

 - 상황에 따라 달라지며, 정규화 수행과 수행하지 않은 결과의 차이가 큰 것이 중요한 포인트임
 - 연속형 데이터를 기계학습 모델링에 적용하기 위해서 변수 간의 상대적 크기 차이를 제거할 필요가 반드시 존재함
 
  - 예시) 
       - 대표적 비지도학습인 군집분석중 Kmeans는 수치형 변수로 적욯하는 알고리즘
       - 변수간 수치적 distance(거리)를 연산하여 유사한 집단으로 묶는 알고리즘
       - 변수가 각자 다른 범위를 가진 경우 유사 집단 구성시, 변수 간 영향력의 차이가 반영되어 올바르지 못한 결과 도출 가능성 존재